In [1]:
# 
import numpy as np
import cv2 # opencv-python which is an Open Source Computer Vision Library
import sys
import os
import matplotlib.pyplot as plt # matplotlib.pyplot which provides a MATLAB-like plotting framework

def close_window():
    cv2.destroyAllWindows()  # Close the window
    cv2.waitKey(1)
    cv2.waitKey(1)
    
def ifNotExistExit(filename):
    if not filename:
        print('filename is None')
        sys.exit()

In [13]:
## GAUSSIAN BLUR
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

## CANNY EDGE DETECTION
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

## REGION OF INTEREST
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(img, mask)

## HOUGH LINES
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

## DRAW LINES
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    if lines is None:
        return
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
## AVERAGE SLOPE
def average_slope(lines):
    if lines is None:
        return [], []
    left_lines = []
    right_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)
            if slope < 0:
                left_lines.append(line)
            else:
                right_lines.append(line)
    return left_lines, right_lines

## AVERAGE LINE
def average_line(lines):
    x = []
    y = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            x += [x1, x2]
            y += [y1, y2]
    if len(x) == 0:
        return None
    return np.polyfit(x, y, 1)

## DRAW LINE
def draw_line(img, line, color=[255, 0, 0], thickness=2):
    if line is None:
        return
    x1 = int((img.shape[0] - line[1]) / line[0])
    x2 = int((img.shape[0] * 0.6 - line[1]) / line[0])
    cv2.line(img, (x1, img.shape[0]), (x2, int(img.shape[0] * 0.6)), color, thickness)
    
## PROCESS IMAGE
def process_image(image):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        blur = gaussian_blur(gray, 5)
        edge = canny(blur, 50, 150)
        imshape = image.shape
        vertices = np.array([[(0, imshape[0]), (imshape[1] * 0.45, imshape[0] * 0.6), (imshape[1] * 0.55, imshape[0] * 0.6), (imshape[1], imshape[0])]], dtype=np.int32)
        roi = region_of_interest(edge, vertices)
        lines = hough_lines(roi, 1, np.pi / 180, 30, 10, 20)
        left_lines, right_lines = average_slope(lines)
        left_line = average_line(left_lines)
        right_line = average_line(right_lines)
        result = np.zeros_like(image)
        draw_line(result, left_line)
        draw_line(result, right_line)
        return result
    except Exception as e:
        print(f"Error in process_image: {e}")
        return image  # Return original image on error

## PROCESS VIDEO
def process_video(input_file, output_file):
    cap = cv2.VideoCapture(input_file)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_file, fourcc, 20.0, (640, 360))
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        result = process_image(frame)
        out.write(result)
    cap.release()
    out.release()
    
## MAIN
def main():
    # Check if running in Jupyter notebook (sys.argv[0] will be empty string)
    if len(sys.argv) <= 1:
        # Running in Jupyter, use direct file path
        print('Running in Jupyter notebook mode')
        
        # Try to find the image in the current directory first
        img_path = "./fig/blue_eyes.png"
        if not os.path.exists(img_path):
            # Try a relative path from the notebook location
            img_path = "../fig/blue_eyes.png"
            if not os.path.exists(img_path):
                print(f'Image not found at {img_path}')
                print(f'Current working directory: {os.getcwd()}')
                return
        
        src = cv2.imread(img_path)
        
        if src is None:
            print(f'Image load failed from {img_path}!')
            return
        
        print(f'Successfully loaded image from {img_path}')
        dst = process_image(src)
        
        cv2.imshow('src', src)
        cv2.imshow('dst', dst)
        cv2.waitKey(0)
        close_window()
    else:
        # Original command-line behavior
        if len(sys.argv) < 3:
            print('Usage: python3 {} <input_file> <output_file>'.format(sys.argv[0]))
            sys.exit()
        input_file = sys.argv[1]
        output_file = sys.argv[2]
        if not os.path.exists(input_file):
            print('File not found: {}'.format(input_file))
            sys.exit()
        process_video(input_file, output_file)
        print('Done')
        
# Run the main function
main()

Usage: python3 /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py <input_file> <output_file>


SystemExit: 

In [ ]:
# Usage Sobel edge detection
src = cv2.imread('./fig/plates.png', cv2.IMREAD_GRAYSCALE)

# dx kernel
dx_kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
print('dx kernel:', dx_kernel)
gx = cv2.filter2D(src, -1, dx_kernel) # cv2.CV_32F

# dy kernel
print('dy kernel:', dx_kernel.T)
gy = cv2.filter2D(src, -1, dx_kernel.T) # cv2.CV_32F


cv2.imshow('src', src)
cv2.imshow('gx', gx)
cv2.imshow('gy', gy)

cv2.waitKey(0)
close_window()


dx kernel: [[-1.  0.  1.]
 [-2.  0.  2.]
 [-1.  0.  1.]]
dy kernel: [[-1. -2. -1.]
 [ 0.  0.  0.]
 [ 1.  2.  1.]]


2025-03-24 20:05:27.055 python[92650:11407762] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-24 20:05:27.055 python[92650:11407762] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
# Usage Sobel edge detection
src = cv2.imread('./fig/son.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
	print('Error: Image not found or unable to load.')
else:
	dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
	dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)
	mag = cv2.magnitude(dx, dy) # cv2.CV_32F
	mag = np.clip(mag, 0, 255).astype(np.uint8) # cv2.CV_8U
	dst = cv2.convertScaleAbs(mag) # cv2.CV_8U
	ret, dst = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY) # cv2.THRESH_BINARY_INV
	
	cv2.imshow('src', src)
	cv2.imshow('mag', mag)
	cv2.imshow('dst', dst)
 
	cv2.waitKey()
	close_window()

In [16]:
## Canny edge detection
src = cv2.imread('./fig/son.jpg', cv2.IMREAD_GRAYSCALE)
if src is None:
    print('Image load failed!')
    sys.exit()
    
dst_canny = cv2.Canny(src, 150, 180) # cv2.CV_8U

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('dst_canny', dst_canny)
cv2.waitKey()
close_window()

In [31]:
## Hough line detection
src = cv2.imread('./fig/checkerboard.png', cv2.IMREAD_GRAYSCALE)
if src is None:
    print('Image load failed!')
    sys.exit()

edges = cv2.Canny(src, 50, 150)
lines = cv2.HoughLinesP(edges, 1, np.pi / 360, 60, minLineLength=10, maxLineGap=20) # cv2.CV_32F
print('lines.shape:', lines.shape) # (n, 1, 2)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

for i in range(lines.shape[0]):
    pt1 = (lines[i][0][0], lines[i][0][1])
    pt2 = (lines[i][0][2], lines[i][0][3])
    cv2.line(dst, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(dst, str(i), pt1, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('edges', edges)
cv2.imshow("dst", dst)

cv2.waitKey()
close_window()

lines.shape: (109, 1, 4)


In [39]:
## Hough Circle detection
src = cv2.imread('./fig/plates.png', cv2.IMREAD_COLOR)
if src is None:
    print('Image load failed!')
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1)
circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 
                           50, param1=200, param2=100, minRadius=50, maxRadius=150)
print('circles.shape:', circles.shape)

dst = src.copy()
if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (int(cx), int(cy)), int(radius), (0, 0, 255), 2, cv2.LINE_AA)
        cv2.circle(dst, (int(cx), int(cy)), 2, (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
close_window()

circles.shape: (1, 7, 3)


In [38]:
circles # 원좌표 (cx, cy, radius)

array([[[479.5 , 147.5 , 100.4 ],
        [204.5 , 333.5 ,  78.6 ],
        [113.5 , 112.5 ,  64.4 ],
        [236.5 , 180.5 ,  61.5 ],
        [668.5 , 231.5 ,  97.3 ],
        [525.5 , 330.5 ,  59.7 ],
        [316.5 ,  76.5 ,  50.25]]], dtype=float32)

In [49]:
## 연결 객체 검출: 레이블링
src = cv2.imread('./fig/keyboard.jpg')
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# mask 하는 이유는 흰색을 255(컴퓨터는 255를 물채로 인식)로 하고 
# 검은색을 0으로 하기 위해서임
_, mask = cv2.threshold(src_gray, 120, 255, cv2.THRESH_BINARY_INV) 
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

# cnt
print('cnt:', cnt)

for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]
    if area <= 200:
        continue
    cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)
    cv2.putText(src, str(i), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

cv2.imshow('src', src)
cv2.imshow('mask', mask)
cv2.imshow('src_gray', src_gray)

cv2.waitKey()
close_window()



cnt: 124
